In [ ]:
import pandas as pd
import json
import numpy as np
from shapely.geometry import MultiLineString, shape
from ast import literal_eval

def str_to_dict(x):
    if 'MultiLineString' in x:
        str_ = x.split('coordinates=')[1].split(', center')[0]
        dict_ = shape(MultiLineString(literal_eval(str_)))
    elif 'Point' in x:
        str_ = x.replace('=', ':')
        str_ = str_.replace("{", "{'")
        str_ = str_.replace("}", "\'}")
        str_ = str_.replace(":", "\':\'")
        str_ = str_.replace("t, ", "t\',\'")
        str_ = str_.replace("\'[", "[")
        str_ = str_.replace("]\'", "]")
        dict_ = literal_eval(str_)
    return dict_

In [13]:
file = '/Users/juliatuvaleva/Documents/marchi/data-29580-2020-02-26.json'

In [14]:
if '.json' in file:
    with open(file, 'r', encoding='cp1251', errors='ignore') as data_file:
        data_loaded = json.load(data_file)
    df = pd.DataFrame(data_loaded)
    output_file = file.split('/')[-1].split('.json')[0]
    
elif '.csv' in file:
    try:
        df = pd.read_csv(file)        
    except:
        df = pd.read_csv(file, sep=';', encoding='1251')
        
    output_file = file.split('/')[-1].split('.csv')[0]
    
else:
    df = pd.read_excel(file, )
    output_file = file.split('/')[-1].split('.xlsx')[0]

print(output_file)

data-29580-2020-02-26


In [15]:
cols_lower = [k.lower() for k in df.columns]

if 'geodata' in cols_lower:
    df = df.dropna(subset = ['geoData'])
    if type(df.loc[0, 'geoData']) == dict:
        df['geometry'] = df['geoData'].apply(lambda x: shape(x))
    else:
        df['geometry'] = df['geoData'].apply(lambda x: shape(str_to_dict(x)))
        
    if 'Longitude_WGS84' in cols_lower:
        df['Lon_WGS84'] = df['Longitude_WGS84']
        df['Lat_WGS84'] = df['Latitude_WGS84']
        df = df.drop(labels = ['geoData', 'Longitude_WGS84', 'Latitude_WGS84'], axis = 1)
    else:
        df = df.drop(labels = ['geoData'], axis = 1)
elif 'geometry' in cols_lower:
    print('data with geo')
else: 
    print('data without geo')

if 'publicphone' in cols_lower:
    if type(df.loc[0,'PublicPhone']) == dict:
        df['PublicPhone'] = df['PublicPhone'].apply(lambda x: list(x[0].values())[0])

In [16]:
df.head()

,global_id,OnTerritoryOfMoscow,NDOC,DDOC,ADR_TYPE,VID,SOSTAD,STATUS,OBJ_TYPE,ADDRESS,...,KAD_ZU,TDOC,L2_TYPE,L2_VALUE,P4,P6,P3,P90,P91,geometry
0,645622140,да,01-41-2050,19.08.2004,Официальный,Адрес,Зарегистрирован в АР,Внесён в ГКН,Здание,"город Москва, Варшавское шоссе, дом 150, строе...",...,[{'KAD_ZU': '77:05:0008003:0011'}],Распоряжение префектуры АО города Москвы,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((37.60137960355606 55.5991142140071, ..."
1,645622141,да,875-В-РП,28.07.2004,Официальный,Адрес,Зарегистрирован в АР,Внесён в ГКН,Здание,"город Москва, Косинская улица, дом 26А",...,[{'KAD_ZU': '-'}],Распоряжение префектуры АО города Москвы,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((37.82795045457843 55.71766099284537,..."
2,645622142,да,3026-р,04.07.2005,Официальный,Адрес,Зарегистрирован в АР,Внесён в ГКН,Здание,"город Москва, Гороховский переулок, дом 21",...,[{'KAD_ZU': '-'}],Распоряжение префектуры АО города Москвы,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((37.66829952083921 55.76624314832979,..."
3,645622163,да,190-В-РП,22.02.2008,Официальный,Адрес,Зарегистрирован в АР,Внесён в ГКН,Здание,"город Москва, Тагильская улица, дом 4, строени...",...,[{'KAD_ZU': '77:03:0002005:0004'}],Распоряжение префектуры АО города Москвы,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((37.75279755616609 55.81824335945878,..."
4,645622174,да,9169,31.10.2003,Официальный,Адрес,Зарегистрирован в АР,Внесён в ГКН,Здание,"город Москва, Ленинградский проспект, дом 73А,...",...,[{'KAD_ZU': '77:09:0005002:0045'}],Распоряжение префектуры АО города Москвы,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((37.51364623922306 55.80362963584685,..."


In [17]:
df.to_csv('/Users/juliatuvaleva/Documents/marchi/datamos/{}_with_geo.csv'.format(output_file), index = False)